# Carrying out PEC on one qubit

This is a demo of PEC carried out on a single qubit. This notebook uses the ideal model coefficients to determine how well it works. The full version would involve learning the model coefficients first and then using those to do PEC.

In [7]:
import numpy as np
from numpy.random import randint
from decimal import Decimal

from qiskit.providers.aer.noise import kraus_error, NoiseModel
from qiskit.quantum_info import Kraus, Pauli
from qiskit import Aer, QuantumCircuit, execute

Inputting the model parameters

In [2]:
T = [Pauli("X"), Pauli("Y"), Pauli("Z")] #These are the terms considered in the model
coeffs = [.03, .01, .05] #These are the corresponding lambdas
omegas = np.multiply(.5, np.add(1, np.exp(np.multiply(-2.0, coeffs)))) #Equation (S9)
overhead = np.exp(2.0*np.sum(coeffs)) #below Eq (5)

Using these parameters to generate a Kraus map representing the error model
$$
\Lambda(\rho) = \prod_{k \in \mathcal{T}}\omega_k\rho+(1-\omega_k)P_k\rho P_k \\
= (\omega_1I\otimes I + (1-\omega_1)P_1^\top \otimes P_1) \circ (\omega_2I\otimes I + (1-\omega_2)P_2^\top \otimes P_2) \circ ... \circ (\omega_NI\otimes I + (1-\omega_N)P_N^\top \otimes P_N)\vert \rho \rangle \rangle
$$

In [3]:
kraus_ops = Kraus(np.identity(2)) #Start with identity map

#Repeatedly compose (w_kI+(1-w_k)P_k) to get error channel as Kraus map
for i in range(len(T)):
    op = Kraus([T[i].to_matrix()*np.sqrt(1-omegas[i]),np.sqrt(omegas[i])*np.identity(2)]);
    kraus_ops = kraus_ops.compose(op);

kraus_error_channel = kraus_error(kraus_ops.data) #create kraus error channel from kraus map
kraus_noise_model = NoiseModel() #create noise model from kraus error channel
kraus_noise_model.add_all_qubit_quantum_error(kraus_error_channel, ['id', 'rx', 'ry', 'rz']) #add noise to all gates
kraus_basis_gates = kraus_noise_model.basis_gates #get noisy basis gates

/home/ben/.local/lib/python3.10/site-packages/qiskit/quantum_info/operators/channel/kraus.py:92: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if isinstance(data, np.ndarray) or np.array(data).ndim == 2:


## Inverse map
$$
\Lambda^{-1}(\rho) = \gamma\prod_{k \in \mathcal{T}}\omega_k \rho - (1-\omega_k)P_k\rho P_k
$$

Sample from the inverse distribution according to the procedure outlined on page 4, section B in the text:
1. Sample the identity with probability $\omega_k$ and $P_k$ otherwise
2. Record the number of times a Pauli was applied
3. Compose the sampled operators together and compose (insert?) into the gate being mitigated
4. Scale the result of this instance by $(-1)^m\gamma$

In [4]:
samples = 10000 #number of circuits to sample
total = 0 #stores the estimator value

gate_to_mitigate = Pauli('Y') #The gate that is being mitigated

backend = Aer.get_backend('qasm_simulator') #backend is a simulator (for now)

#I wanted to be able to control the precision of the rng
def rand_precision(prec_digits):
    return Decimal(randint(0, 10**prec_digits))/10**prec_digits

for i in range(samples):
    m = 1 #store the sign, alternating when applying non-id Pauli
    op = gate_to_mitigate #start with the gate to be mitigated
    
    for (j,Pk) in enumerate(T):
        if rand_precision(10) < 1-omegas[j]: #with prob 1-w_k apply pauli (in software)
            m*=-1 #alternate sign
            op = op.compose(Pk)
            
    qc = QuantumCircuit(1,1) 
    qc.append(op.to_instruction(),[0]) #apply combination of paulis in circuit
    qc.measure(0,0)
    
    #run one shot of this circuit
    counts = execute(qc, backend, noise_model=kraus_noise_model, 
                     basis_gates = kraus_basis_gates, shots = 1,
                    optimization_level = 0).result().get_counts()
    total += (counts.get('0',0)-counts.get('1',0))*m #scale the results accordingly

Get the mitigated value and the unmitigated value, then compare them to the actual value

In [5]:
#get the unmitigated value
qc = QuantumCircuit(1,1)
qc.append(gate_to_mitigate, [0])
qc.measure(0,0)
counts = execute(qc, backend, noise_model=kraus_noise_model, 
                     basis_gates = kraus_basis_gates, shots = samples,
                    optimization_level = 0).result().get_counts()

#compare the mitigated, unmitigated, and actual values
print("Unmitigated value: ", (counts.get('0',0)-counts.get('1',0))/samples)
print("Mitigated value: ", total*overhead/samples)
print("Ideal value: ", "%.2f"%{True:1, False:-1}[gate_to_mitigate.compose(Pauli('Z')) == Pauli('Z').compose(gate_to_mitigate)])

Unmitigated value:  -0.9164
Mitigated value:  -1.0003948286245847
Ideal value:  -1.00
